In [32]:
import numpy as np
import pandas as pd
import mne
import sys
import platform
import os

if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')
from base.files_in_out import getListOfFiles,GetFiles


## Code to plot and save the figures of the cluster centers

In [2]:
cluster_epo= mne.read_epochs('ana/prestate/clus_center/wholegfp/cfa_n_vep_prestate_wholegfp_cluster_centers_5_epo.fif')
info_epo=mne.read_epochs('preproc/g01/g01_prestate/g01_n_tsk_cfa_vep_gfp_epo.fif').info

Reading Z:\BBC\WP1\data\EEG\tsk\ana\prestate\clus_center\wholegfp\cfa_n_vep_prestate_wholegfp_cluster_centers_5_epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =       0.00 ...       0.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
5 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading Z:\BBC\WP1\data\EEG\tsk\preproc\g01\g01_prestate\g01_n_tsk_cfa_vep_gfp_epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =       0.00 ...       0.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Adding metadata with 40 columns
415 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [2]:
%matplotlib qt

import matplotlib.pyplot as plt
for n,topo in enumerate(cluster_epo.get_data()):
    map_data=topo.flatten()
    if n ==3:
        map_data=map_data*-1
    mne.viz.plot_topomap(map_data,info_epo)
    plt.savefig(f'ana/prestate/clus_center/wholegfp/whole_gfp_clus5_map{n}.svg',format='svg',dpi=900)

NameError: name 'cluster_epo' is not defined

## Code to plo the figures of the gev and time coverage means
obtains box plots

In [33]:

import matplotlib.pyplot as plt
import seaborn as sns


In [34]:
%load_ext autoreload

%autoreload 2

plt.style.use('default')
import matplotlib as mpl
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# simple function to read df
def read_df(phy_phase, micro_type,n_clus):
    param_path='ana/prestate/param df/'
    param_fn=f'{micro_type}/{phy_phase}_{n_clus}_clusters_tsk_n_prestate_{micro_type}.feather'
    param_fp=param_path+param_fn
    df=pd.read_feather(param_fp)
    # convert time cov in percentage for display
    df['timecov_']=df['timecov_']*100
    return df


In [36]:
n_clus=5
phy_phase='cardiac_phase'
micro_type='wholegfp'



In [37]:
df=read_df(phy_phase,micro_type,n_clus)

# filter dependant variable to loop into later
dv_columns=['timecov_','gev_']

# define path to save output
figpath='ana/results_report/images/prestate/'

In [16]:
%matplotlib qt

## Main boxplot function

In [55]:
from matplotlib.patches import Patch
def plot_boxplot(ax, df,positions, phy_phase , box_width=0.5, linewidth=2, fontsize=14):
    """
    Function to generate and style boxplots with the given data.
    Arguments:
    - ax: the axis where the boxplot will be drawn
    - df: the dataframe for the boxplot
    - positions: positions for the boxes
    - palette: color palette for styling
    - fill_styles: fill styles for boxes
    - order: order of the columns to plot boxplots
    - box_width: width of the boxes (default is 0.5)
    - linewidth: the width of the lines for all components (default is 2)
    - fontsize: size of the font to unify it
    """
        # define variables that change depending on cardiac or rsp condition
    print(df.columns[0][0])
    if 'aware' in df.columns[0][0]:

        cond_grouping='awareness'
    else:
        cond_grouping='phy_cond'

    if phy_phase=='cardiac_phase':
        # colors to use for the plots
        palette=dict(sys='tab:orange',dia='#4C72B0')
        #order of columns of the df to reorder boxplots
        if cond_grouping=='awareness':
            order=[('aware','sys'), ('aware','dia'), ('unaware','sys'), ('unaware','dia')]
        elif cond_grouping=='phy_cond':
            order=[('sys', 'aware'), ('sys','unaware'), ('dia', 'aware'), ('dia','unaware')]
        else:
           raise ValueError('the grouping condition does not exhist: try valid awareness or phy_cond')
        # name of the x labels
        xtick=['systole','diastole']
    elif phy_phase=='rsp_phase':
        # colors to use for the plots
        palette=dict(inh='g',exh='tab:pink')
        #order of columns of the df to reorder boxplots
        if cond_grouping=='awareness':
            order=[('aware','inh'), ('aware','exh'), ('unaware','inh'), ('unaware','exh')]
        elif cond_grouping=='phy_cond':
            order=[('inh', 'aware'), ('inh','unaware'), ('exh', 'aware'), ('exh','unaware')]
        else:
           raise ValueError('the grouping condition does not exhist: try valid awareness or phy_cond')
        # name of the x labels
        xtick=['inhalation','exhalation']


    # styles to plot to indicate different aware
    fill_styles=dict(aware='filled',unaware='empty')
    df= df[order]
    boxplot = ax.boxplot(df, positions=positions, patch_artist=True, widths=box_width)

    # Prepare the color palette and fill styles based on the columns
    colors = [palette[k] for col in df.columns for k, item in palette.items() if k in col]
    double_color = [item for item in colors for _ in range(2)]
    fills = [fill_styles[k] for col in df.columns for k, item in fill_styles.items() if k in col]
    # Prepare handles for legend
    handles = []

    # Style boxplot components (boxes, fliers, whiskers, etc.) with linewidth
    for patch, outlier, color, fill, column in zip(boxplot['boxes'], boxplot['fliers'], colors, fills, df.columns):

        label=f'{column[0]} {column[1]}'

        patch.set_edgecolor(color)
        patch.set_linewidth(linewidth)
        outlier.set_markersize(5)
        outlier.set_markeredgewidth(linewidth)
        outlier.set_markeredgecolor(color)
        outlier.set_markerfacecolor('white')  # Optional, change as per your preference
        patch.set_facecolor(color if fill == 'filled' else 'white')
        # Create legend handles for filled and empty boxes with full rectangle
        if fill == 'filled' :
            handles.append(Patch(facecolor=color, label=label, edgecolor=color,linewidth=linewidth))
        elif fill == 'empty':
            handles.append(Patch(facecolor='white', label=label, edgecolor=color,linewidth=linewidth))


    # Style whiskers and caps with linewidth
    for whisker, cap, color in zip(boxplot['whiskers'], boxplot['caps'], double_color):
        whisker.set_color(color)
        cap.set_color(color)
        whisker.set_linewidth(linewidth)
        cap.set_linewidth(linewidth)

    # Style the medians with linewidth
    for median in boxplot['medians']:
        median.set_color('black')
        median.set_linewidth(linewidth)

        # Add legend if legend labels are provided

    ax.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.5,-0.1),fontsize=fontsize)
    plt.tight_layout()
    return xtick





## Code to obtain all pictures for all conditions

In [61]:

# Main loop to generate plots for maps and dv_columns
for maps in df['map'].unique():
    # Select map to loop into
    df_map = df[df['map'] == maps]

    for dv_col in dv_columns:
        # Define df with awa and phy cond as columns
        df_awa = df_map.pivot_table(index=['g_num'], values=[dv_col], columns=['awareness', 'phy_cond'])[dv_col]
        df_phy_cond = df_map.pivot_table(index=['g_num'], values=[dv_col], columns=['phy_cond', 'awareness'])[dv_col]

        # Create the figure and axes
        fig, [ax1, ax2] = plt.subplots(1, 2,figsize=(12,8))

        # Set positions for the boxplots
        positions = [1.5, 2.5, 4.5, 5.5]
        font_size=14

        # Plot first boxplot for awareness


        plot_boxplot(ax1, df_awa, positions, phy_phase , box_width=0.9,  linewidth=2.5,fontsize=font_size)


        yaxisname=[]
        if dv_col=='timecov_':
            yaxisname='time coverage (%)'
            #ylim=(4,40)
        elif dv_col=='gev_':
            yaxisname='GEV'
            #ylim=(0,0.25)

        ax1.set_ylabel(yaxisname,fontsize=font_size)
        ax1.set_xticks([1.5,4.5], minor= False)
        ax1.set_xticklabels(['aware','unaware'],fontsize=font_size)
        #ax1.set_ylim(ylim)
        ax1.tick_params(axis='both', labelsize=font_size)
        title= f'Map: {maps}'
        ax1.set_title(title)



        # Plot second boxplot for phy_cond

        xtick_2=plot_boxplot(ax2, df_phy_cond, positions, phy_phase, box_width=0.9, linewidth=2.5,fontsize=font_size)

        ax2.set_xticks([1.5,4.5], minor= False)
        ax2.set_xticklabels(xtick_2,fontsize=font_size)
        #ax2.set_ylim(ylim)
        ax2.set_ylabel(yaxisname,fontsize=font_size)
        ax2.tick_params(axis='both', labelsize=font_size)

        # save output figure
        figname=f'map{maps}_{dv_col}{phy_phase[:3]}.svg'
        figpathname=figpath+figname
        #plt.savefig(figpathname,dpi=1000,format='svg')

aware
dia
aware
dia
aware
dia
aware
dia
aware
dia
aware
dia
aware
dia
aware
dia
aware
dia
aware
dia


## Plot figure for article

In [26]:
n_clus=5

micro_type='wholegfp'
df_card=read_df('cardiac_phase',micro_type,n_clus)
df_rsp=read_df('rsp_phase',micro_type,n_clus)


In [27]:


# filter dependant variable to loop into later
dv_columns=['timecov_','gev_']

# define path to save output
figpath='ana/results_report/images/prestate/'

for the picture we need to select following conditions for each of the 4 suplots:
Map= map to select
phy_cond= whether cardiac or respiratory phase
dv_col= dependent variable, either gev or time coverage
cond_grouping: whether to plot awareness or the physical condition on the X axis
plt1: map=[ Map 1, phy_cond= rsp, cond_grouping= phy cond,dv_col= gev]
plt2: map=[ Map 1, phy_cond= card, cond_grouping= phy cond, dv_col= gev]

plt3: map=[ Map 0, phy_cond= rsp, cond_grouping= awareness,dv_col= time coverage]
plt3: map=[ Map 0, phy_cond= card, cond_grouping= awareness,dv_col= time coverage,]


In [28]:
def filter_df(df,map,cond_grouping,dv_col):
    """
    Function to filter dataset to be ready to plot according to selected variable.
    The physical condition (either card or rsp) has to be preselected before to avoid keep reloading df
    Arguments:
    - df: the dataframe either for card or rsp
    - map: which map number to plot
    - cond_grouping: whether the boxplot will be grouped by awareness or physical condition
    - dv col: determines dv whether gev or time coverage
    """
    df=df[df['map'] == map]
    if cond_grouping=='awareness':
        df = df.pivot_table(index=['g_num'], values=[dv_col], columns=['awareness', 'phy_cond'])[dv_col]
    elif cond_grouping=='phy_cond':
        df = df.pivot_table(index=['g_num'], values=[dv_col], columns=['phy_cond', 'awareness'])[dv_col]
    return df




In [76]:
#

# Define df with awa and phy cond as columns
df_1=filter_df(df_rsp,1,'phy_cond','gev_')
df_2=filter_df(df_card,1,'phy_cond','gev_')

df_3=filter_df(df_rsp,0,'awareness','timecov_')
df_4=filter_df(df_card,0,'awareness','timecov_')
# Create the figure and axes
fig, [ax1, ax2,ax3,ax4] = plt.subplots(1, 4,figsize=(18,8))

# Set positions for the boxplots
positions = [1, 2, 3.5, 4.5]
font_size=14

xlim=(0.3,5.2)
yaxisname_time='time coverage (%)'
ylim_time=(4,40)

yaxisname_gev='GEV'
ylim_gev=(0,0.26)
# Plot first boxplot for awareness


xtick_1=plot_boxplot(ax1, df_1, positions, 'rsp_phase' , box_width=0.9,  linewidth=2.5,fontsize=font_size)


ax1.set_ylabel(yaxisname_gev,fontsize=font_size)
ax1.set_xticks([1.5,4.5], minor= False)
ax1.set_xticklabels(xtick_1,fontsize=font_size)
ax1.set_ylim(ylim_gev)
ax1.tick_params(axis='both', labelsize=font_size)
ax1.set_xlim(xlim)

xtick_2=plot_boxplot(ax2, df_2, positions, 'cardiac_phase' , box_width=0.9,  linewidth=2.5,fontsize=font_size)


ax2.set_ylabel(yaxisname_gev,fontsize=font_size)
ax2.set_xticks([1.5,4.5], minor= False)
ax2.set_xticklabels(xtick_2,fontsize=font_size)
ax2.set_ylim(ylim_gev)
ax2.tick_params(axis='both', labelsize=font_size)
ax2.set_xlim(xlim)

plot_boxplot(ax3, df_3, positions, 'rsp_phase' , box_width=0.9,  linewidth=2.5,fontsize=font_size)


ax3.set_ylabel(yaxisname_time,fontsize=font_size)
ax3.set_xticks([1.5,4.5], minor= False)
ax3.set_xticklabels(['aware','unaware'],fontsize=font_size)
ax3.set_ylim(ylim_time)
ax3.tick_params(axis='both', labelsize=font_size)
ax3.set_xlim(xlim)

plot_boxplot(ax4, df_4, positions, 'cardiac_phase' , box_width=0.9,  linewidth=2.5,fontsize=font_size)

ax4.set_ylabel(yaxisname_time,fontsize=font_size)
ax4.set_xticks([1.5,4.5], minor= False)
ax4.set_xticklabels(['aware','unaware'],fontsize=font_size)
ax4.set_ylim(ylim_time)
ax4.tick_params(axis='both', labelsize=font_size)
ax4.set_xlim(xlim)


# save output figure
figname=f'figure_2_raw.svg'
figpathname=figpath+figname
plt.savefig(figpathname,dpi=1000,format='svg')

exh
dia
aware
aware
